In [ ]:
import numpy as np 
import pandas as pd 
import json
import csv
import os
from pandas import json_normalize 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

In [ ]:
from google.colab import files
files.upload()

! mkdir  ~/.kaggle
!cp kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle.json


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d yelp-dataset/yelp-dataset

100% 4.06G/4.07G [00:30<00:00, 157MB/s]
100% 4.07G/4.07G [00:30<00:00, 143MB/s]


In [ ]:
from zipfile import ZipFile
file_name = 'yelp-dataset.zip'
with ZipFile(file_name,'r') as zip:
   zip.extractall()
print('Done')

Done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive"

 202-Individual-Project-Report.gdoc
 255
 255_project
 277-team-STRIKERS-final-ppt.gslides
 appsheet
'Architecture Diagram.png'
 AWS-Architecture-Icons-Deck_For-Light-BG_04302022.gslides
'AWS Services and Tools Utilized - Documentation.gdoc'
 ChaitraRaghunath_Resume.gdoc
 ChaitraRaghunath_Resume.pdf
'CMPE 202--'
 CMPE-202-Patterns-combined.pdf
'cmpe 255'
 CMPE-255-Final-Project-Report.gdoc
 CMPE_255_Project_Presentation.pptx
 CMPE-255-Project-Proposal.gdoc
'CMPE 277 '
'CMPE 277 Smartphone Application Development (Android).gdoc'
 CMPE-277-SmartphoneApplicationDevelopment-.gdoc
 CMPE-281-Project-1-Documentation.gdoc
 CMPE-282-hw-7-security
'Colab Notebooks'
 converted_data
'Copy of DesignSpec_Patrick.docx'
'Copy of FINAL_NLPv1.ipynb'
'Copy of SJSU SE CE Course Classification.gdoc'
'Copy of UML Seq Diagram.gdoc'
'Cover Letter Template.gdoc'
'ENGR 200W - Analyzing the Audience.gdoc'
'Enterprise Final Project Report.docx'
 EzCloudStore.zip
 EzStore.zip
 Gayle_McDowell_CareerCup_Sample_Resum

In [ ]:
#review_data = pd.read_json("yelp_academic_dataset_review.json", lines=True)

Selecting only business_id and text from reviews (Feature Selection) and separating them into high risk reviews and remaining reviews in other dataframe

In [ ]:
with open("yelp_academic_dataset_review.json") as reviews:
    
    words = ["discomfort","ache","death","allergy","diarrhoea", "stomach pain", "loose motions", "loose motion", "food poisoning", "sick", "vomitting", "nausea", "throw up", "threw up", "unhygienic", "pain", "flu", "stale", "poisioning"]
    count = 0 
    
    highRiskReviews = pd.DataFrame(columns=['business_id', 'text'])
    restReviews = pd.DataFrame(columns=['business_id', 'text'])
    
    # Iterate through the reviews and save high risk reviews to highRiskReviews
    for index,review in enumerate(reviews):
        reviewData = json.loads(review)
        
        if any(word in reviewData['text'] for word in words) and reviewData['stars'] < 2:
            dict_rec = {'business_id': reviewData['business_id'], 'text': reviewData['text']}
            highRiskReviews = highRiskReviews.append(dict_rec, ignore_index=True)
        
        if np.random.randint(1,1000) == 1 and not any(word in reviewData['text'] for word in words):
            dict_rec = {'business_id': reviewData['business_id'], 'text': reviewData['text']}
            restReviews = restReviews.append(dict_rec, ignore_index=True)
            count += 1
        
    print(highRiskReviews.info())
    print(restReviews.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101094 entries, 0 to 101093
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  101094 non-null  object
 1   text         101094 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6716 entries, 0 to 6715
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  6716 non-null   object
 1   text         6716 non-null   object
dtypes: object(2)
memory usage: 105.1+ KB
None


In [ ]:
def formatDataFromID(business_ids, highRisk = 0):
    
    X = pd.DataFrame()

    with open("yelp_academic_dataset_business.json") as businesses:
        
        for business in businesses:

            businessData = json.loads(business)

            if businessData['business_id'] in business_ids:
                if businessData['categories']:
                    if 'Food' in businessData['categories'] or 'Restaurants' in businessData['categories']:

                        # Drop columns that are unlikely to be useful features (think about dropping review count and number of stars as this will leak data)
                        columnsToDrop = ["hours", "name", "address", "city", "state", "postal_code", "latitude", "longitude", "is_open", "BusinessParking", "GoodForMeal"]
                        [businessData.pop(column, None) for column in columnsToDrop]

                        # Add in category data, represented as a string list
                        categoryColumns = businessData["categories"].split(",")
                        for category in categoryColumns:
                            businessData['category_' + category.strip()] = 1

                        # Add in attribute data, represented as a string dict
                        if businessData["attributes"]:
                            for key,value in businessData["attributes"].items():
                                if key == "Ambience":
                                    if businessData["attributes"]["Ambience"] != "None":
                                        ambienceType = businessData["attributes"].get(key)
                                        for key,value in eval(ambienceType).items():
                                            if value == 0:
                                                businessData['ambience_' + key] = 0
                                            else:
                                                businessData['ambience_' + key] = 1

                        # Assign the target variable (whether the restaurant has a high risk of food poisoning)
                        businessData['highRisk'] = highRisk

                        X = X.append(json_normalize(businessData, sep='_'), ignore_index=True)
        return X


In [ ]:
highRiskBusinesses = highRiskReviews['business_id'].tolist()
normalBusinesses = restReviews['business_id'].tolist()

X_highRisk = formatDataFromID(highRiskBusinesses, 1)
X_normal = formatDataFromID(normalBusinesses)

In [ ]:
X_highRisk.head()

,business_id,stars,review_count,categories,category_Vietnamese,category_Food,category_Restaurants,category_Food Trucks,ambience_touristy,ambience_hipster,...,category_Hypnosis/Hypnotherapy,category_Traditional Chinese Medicine,category_Bike tours,category_Bike Rentals,category_Countertop Installation,category_Security Systems,category_Keys & Locksmiths,category_Serbo Croatian,category_Towing,category_Go Karts
0,eEOYSgkmpB90uNA7lDOMRA,4.0,10,"Vietnamese, Food, Restaurants, Food Trucks",1.0,1.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0bPLkL0QhhPO5kt1_EXmNQ,4.5,100,"Food, Delis, Italian, Bakeries, Restaurants",NaN,1.0,1.0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MUTTqe8uqyMdBl186RmNeA,4.0,245,"Sushi Bars, Restaurants, Japanese",NaN,NaN,1.0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9OG5YkX1g2GReZM0AskizA,2.5,339,"Restaurants, Italian",NaN,NaN,1.0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tMkwHmWFUEXrC9ZduonpTg,4.0,161,"Restaurants, Japanese, Seafood",NaN,NaN,1.0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
type(X_highRisk)

pandas.core.frame.DataFrame

In [ ]:
print(X_highRisk.shape)
print(X_normal.shape)

(23096, 676)
(4265, 449)


Merging high Risk and normal data together to be fed into the algorithm

In [ ]:
X = pd.concat([X_highRisk, X_normal], ignore_index=True)

In [ ]:
X.shape

(27361, 687)

Encoding Business_id for easy understanding 

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X['business_id'] = le.fit_transform(X['business_id'])

In [ ]:
X.shape

(27361, 687)

In [ ]:
X_clean = X.drop(['attributes_Ambience', 'categories', 'category_Restaurants', 'attributes_BusinessParking','attributes_BikeParking', 'attributes_Music'], axis=1)

cols_with_missing = [col for col in X.columns if X[col].isnull().sum() > X.shape[0]*0.9]
X_clean = X_clean.drop(cols_with_missing, axis=1)

X_clean = X_clean.fillna(0)

Only considering those businesses which have atleast 2 reviews of food poisoning

In [ ]:
counts = X_clean['business_id'].value_counts()
res = X_clean[X_clean['business_id'].isin(counts[counts > 1].index)]

In [ ]:
res.shape

(5420, 48)

Splitting the dataset into train and test set

In [ ]:
Y = res['highRisk']
X = res.iloc[:, res.columns != 'highRisk']

print(X.shape)
print(Y.shape)
print(X.head())

(5420, 47)
(5420,)
    business_id  stars  review_count  category_Food  ambience_touristy  \
3          4106    2.5           339            0.0                0.0   
19         7484    4.0           389            1.0                0.0   
23        22989    4.0           129            0.0                0.0   
31        11362    4.0          2404            1.0                0.0   
33        13284    4.0           182            0.0                1.0   

    ambience_hipster  ambience_romantic  ambience_divey  ambience_intimate  \
3                0.0                1.0             0.0                0.0   
19               1.0                0.0             0.0                0.0   
23               0.0                0.0             0.0                0.0   
31               0.0                0.0             0.0                0.0   
33               1.0                0.0             1.0                0.0   

    ambience_trendy  ...  category_American (Traditional)  category

In [ ]:
objects = (X.dtypes == 'object')
object_cols = list(objects[objects].index)

# Drop high cardinality objects
low_cardinality_cols = [col for col in object_cols if X[col].nunique() < 10]

# Columns to be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

# Change the column data to be strings, this doesn't matter as they will be OH encoded
for col in object_cols:
    X[col] = X[col].astype(str)

X = X.drop(high_cardinality_cols, axis=1)

# OH encode the data
OH_encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(X[low_cardinality_cols]))

# Put the index back in
OH_cols.index = X.index

# Drop the columns that have been OH encoded
num_X = X.drop(low_cardinality_cols, axis=1)
OH_X = pd.concat([num_X, OH_cols], axis=1)

<ipython-input-23-9f7d16a869fa>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(str)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(OH_X, Y, test_size=0.3)

In [ ]:
import xgboost as xgb

model = xgb.XGBClassifier(n_estimators=500)
model.fit(X_train, Y_train,early_stopping_rounds=10, eval_set=[(X_test, Y_test)], verbose=False)
predictions = model.predict(X_test)
print("Mean absolute error: {:.2f}".format(mean_absolute_error(predictions, Y_test)))
print("Score for XGBoost model is {:.2f}% ".format(model.score(X_test, Y_test)*100))

Mean absolute error: 0.54
Score for XGBoost model is 46.31% 


In [ ]:
from sklearn import tree

DT = tree.DecisionTreeClassifier(criterion='entropy', random_state=0)
DT.fit(X_train, Y_train)
y_pred = DT.predict(X_test)
print("Decision Tree")
print('Accuracy Score: {:.2f}%'.format(metrics.accuracy_score(Y_test,y_pred)*100))
print('Confusion Matrix: ',metrics.confusion_matrix(Y_test,y_pred))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


Decision Tree
Accuracy Score: 14.76%
Confusion Matrix:  [[175 656]
 [730  65]]


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=100, criterion='gini')
RF.fit(X_train, Y_train)
y_pred = RF.predict(X_test)
print('For Random Forest algorithm')
print('Accuracy Score: {:.2f}%'.format(metrics.accuracy_score(Y_test,y_pred)*100))
print('Confusion Matrix: ',metrics.confusion_matrix(Y_test,y_pred))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


For Random Forest algorithm
Accuracy Score: 14.76%
Confusion Matrix:  [[133 698]
 [688 107]]


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
